# Document Classfication
<hr>

* 문서가 주어졌을 때 해당 문서의 범주를 분류하는 과제

Ex) input : 뉴스, output : 정치, 경제, 연예 등 (범주 맞춤)
<br><br>

## NSMC Sentiment Analysis
<hr>

1. 입력 문장에 CLS, SEP 토큰을 붙인다.(CLS, SEP -> 문장 시작과 끝을 알리는 스페셜 토큰)
2. BERT 모델에 입력 ->  문장 수준의 벡터(pooler_output)을 뽑느다.
3. 벡터에 작은 추가 모듈을 통해 모델의 전체 출력이 [긍정확률, 부정확률]로 맞춘다.
<br>

## TASK Module
<hr>
<b>pooler_output</b> 벡터 뒤에 붙는 추가 모듈의 구조는 다음 그림과 같다.
우선 <b>pooler_output</b>  벡터에 드롭아웃을 적용 
<br>드롭아웃을 적용한다는 의미는 그림에서 입력 벡터의 768개 각 요솟값 가운데 일부를 랜덤으로 0으로 바꿔 이후 계산에 포함하지 않도록 한다.
<br>
<img src="task.png">
출처 : ratsgo's NLPBOOK
<br>

그다음 가중치 행렬을 곱해 pooler_output을 분류해야 할 범주 수만큼의 차원을 갖는 벡터로 변환.
<br>만일 pooler_output 벡터가 768차원이고 분류 대상 범주 수가 2개(긍정, 부정)라면 가중치 행렬 크기는 (768,2) 
<br>여기에 소프트맥스 함수를 취하면 모델의 최종 출력
<br>
이렇게 만든 모델의 최종 출력과 정답 레이블을 비교해 모델 출력이 정답 레이블과 최대한 같아지도록 태스크 모듈과 BERT 레이어를 포함한 모델 전체를 업데이트<br>


In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import evaluate
from Korpora import Korpora


In [2]:
import torch

# MPS 지원 여부 확인
if torch.backends.mps.is_available():
    device = torch.device("mps")  # MPS 설정
    print("MPS 지원됨. GPU 사용 가능!")
else:
    device = torch.device("cpu")  # CPU로 설정
    print("MPS 사용 불가. CPU 사용 중!")

MPS 지원됨. GPU 사용 가능!


In [3]:
# NSMC 데이터 다운로드
Korpora.fetch("nsmc", force_download=True)

# 데이터 경로 설정
train_data_path = "../data/nsmc/ratings_train.txt"
test_data_path = "../data/nsmc/ratings_test.txt"

# 데이터셋 로드 및 변환
def load_nsmc_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()[1:]  # 첫 줄(헤더) 제거
    texts, labels = [], []
    for line in lines:
        parts = line.strip().split("\t")
        if len(parts) != 3:
            continue
        text, label = parts[1], int(parts[2])  # 리뷰 내용과 감성 라벨 (0: 부정, 1: 긍정)
        texts.append(text)
        labels.append(label)
    return {"text": texts, "label": labels}


[nsmc] download ratings_train.txt: 14.6MB [00:01, 8.30MB/s]                     
[nsmc] download ratings_test.txt: 4.90MB [00:00, 7.25MB/s]                      


In [4]:

from datasets import Dataset
train_dataset = Dataset.from_dict(load_nsmc_data(train_data_path))
test_dataset = Dataset.from_dict(load_nsmc_data(test_data_path))


# 토크나이징
model_name = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
from torch.utils.data import DataLoader

# 모델 설정
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.to(device)

# 평가 메트릭 수정 (evaluate 라이브러리 사용)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)


from transformers import TrainingArguments

train_dataloader = DataLoader(
    tokenized_train,
    batch_size=16,  # MPS에서는 batch_size=8~16 추천
    shuffle=True,
    num_workers=0,  # Mac에서는 0~2가 적절함
    pin_memory=False,  # MPS에서는 필요 없음
)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,  # MPS에서는 8~16 추천
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    fp16=False,  # MPS에서는 반드시 False
    bf16=False,  # bfloat16도 지원 안 됨
    gradient_accumulation_steps=2,  # 작은 배치 크기 해결책
    load_best_model_at_end=True,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/homebrew/anaconda3/envs/myenv/lib/python3.9/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


	•	학습 관련 설정을 정의하는 부분
	•	output_dir="./results" → 모델 가중치 저장 경로
	•	evaluation_strategy="epoch" → 매 epoch 마다 평가
	•	learning_rate=5e-5 → BERT 모델의 기본 학습률(learning rate)
	•	per_device_train_batch_size=32 → GPU 사용 가능하면 배치 크기 32, 아니면 4
	•	num_train_epochs=3 → 총 3 epoch 학습
	•	save_strategy="epoch" → 매 epoch마다 모델 가중치 저장
	•	logging_dir="./logs" → 로그 저장 경로

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
# 학습 실행
trainer.train()

/var/folders/lt/dt43ws7s4_z4rrsjwvzmwz740000gn/T/ipykernel_31211/169967503.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

	•	Trainer를 사용해 모델 학습을 진행하는 코드
	•	model=model → 사용할 BERT 모델
	•	args=training_args → 학습 설정 (TrainingArguments)
	•	train_dataset=tokenized_train → 훈련 데이터셋
	•	eval_dataset=tokenized_test → 검증 데이터셋
	•	tokenizer=tokenizer → BERT 토크나이저
	•	compute_metrics=compute_metrics → 평가 메트릭 (정확도)
	•	마지막으로 trainer.train()을 실행하면 모델 파인튜닝이 진행

<hr>


In [ ]:
#코드 4-4 모델 환경 설정
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My drive/nlpbook/checkpoint-doccls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [ ]:
#코드 4-7 말뭉치 내려받기
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

In [ ]:

#코드 4-8 토크나이저 준비
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [ ]:
#코드 4-9 학습 데이터셋 구축
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)


In [ ]:
#코드 4-10 학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers
)

In [ ]:
#코드 4-11 평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

## 모델 불러오기
<hr>
이제 모델을 초기화하는 다음 코드를 실행. 
코드 4-4에서 <b>pretrained_model_name을 beomi/kcbert-base</b>로 지정했으므로 프리트레인을 마친 <b>BERT</b>로 <b>kcbert-base</b>를 사용합니다. 
<br>모델을 초기화하는 코드에서 <b>BertForSequenceClassification</b>은 프리트레인을 마친 BERT 모델 위에 <4-1>절에서 설명한 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스
<br>이 클래스는 허깅페이스에서 제공하는 transformers 라이브러리에 포함.


In [ ]:
#코드 4-12 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

허깅페이스 모델 허브에 등록된 모델이라면 별다른 코드 수정 없이 <b>kcbert-base</b> 이외에 다른 언어 모델을 사용할 수 있다.<br> 
예를 들어 bert-base-uncased 모델은 구글이 공개한 다국어 BERT 모델
<br>코드 4-4에서 pretrained_model_name에 이 모델명을 입력하면 해당 모델을 쓸 수 있다. 
<br>허깅페이스에 등록된 모델 목록은 huggingface.co/models에서 확인할 수 있다.
<br>아울러 코드 4-8, 4-12에는 똑같은 모델 이름을 입력해야 한다.

## 모델 학습시키기
<hr>
파이토치 라이트닝(pytorch lighting)이 제공하는 LightingModule 클래스를 상속받아 태스크(task)를 정의. 
<br>태스크에는 다음 그림처럼 모델과 옵티마이저, 학습 과정 등이 정의돼 있다.

<그림 4-6 TASK의 역할>
<img src="task2.jpeg">
출처 : ratsgo's NLPBOOK<br>
다음 코드를 실행하면 문서 분류용 태스크를 정의할 수 있습니다. 
<br>코드 4-4에서 만든 학습 설정(args)과 코드 4-12에서 준비한 모델(model)을 ClassificationTask에 주입합니다. 
<br><b>ClassificationTask</b>에는 옵티마이저(optimizer), 러닝 레이트 스케줄러(learning rate scheduler)가 정의돼 있는데, 옵티마이저로는 아담(Adam), 러닝 레이트 스케줄러로는 ExponentialLR을 사용합니다.

In [ ]:
#코드 4-13 TASK 정의
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

모델 학습 과정은 눈을 가린 상태에서 산등성이를 한 걸음씩 내려가는 과정에 비유 가능. 
<br> ExponentialLR은 현재 에포크의 러닝 레이트를 '이전 에포크의 러닝 레이트  gamma'로 스케줄.
<br>
ratsgo.github.io/nlpbook/docs/doc_cls/detail
<br>
## Inference
* 학습을 마친 모델로 실제 과제를 수행하는 행위나 그 과정

